- change fold to 5 again
- incorporate sum of moa as variable
- install lgb

In [1]:
import os
import sys
import warnings
import numpy as np
import pandas as pd 
import lightgbm as lgb
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, StratifiedKFold
from category_encoders import CountEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import log_loss, mean_squared_error
from sklearn import preprocessing
from sklearn.multioutput import MultiOutputClassifier
sys.path.append('../input/multilabelstraifier/')
from ml_stratifiers import MultilabelStratifiedKFold
warnings.filterwarnings('ignore')

In [2]:
SEED = 42
NFOLDS = 5
np.random.seed(SEED)
DATA_DIR = '/kaggle/input/lish-moa/'

In [3]:
train = pd.read_csv(DATA_DIR + 'train_features.csv')
targets = pd.read_csv(DATA_DIR + 'train_targets_scored.csv')

test = pd.read_csv(DATA_DIR + 'test_features.csv')
sub = pd.read_csv(DATA_DIR + 'sample_submission.csv')

In [4]:
target_feats = [ i for i in targets.columns if i != "sig_id"]
g_feats = [i for i in train.columns if "g-" in i]
c_feats = [i for i in train.columns if "c-" in i]

In [5]:
noncons_train_index = train[train.cp_type=="ctl_vehicle"].index
cons_train_index = train[train.cp_type!="ctl_vehicle"].index
noncons_test_index = test[test.cp_type=="ctl_vehicle"].index
cons_test_index = test[test.cp_type!="ctl_vehicle"].index

# Preprocess

In [6]:
# normalization by ctl group
train_ctl = train[train.index.isin(noncons_train_index)].copy().reset_index(drop=True)
test_ctl = test[test.index.isin(noncons_test_index)].copy().reset_index(drop=True)
ctl_df = pd.concat([train_ctl, test_ctl])

ctl_group_data = ctl_df.groupby(["cp_dose", "cp_time"]).agg({"mean"}).reset_index()
mean_g_feats = ["mean-" + i for i in g_feats]
mean_c_feats = ["mean-" + i for i in c_feats]
columns = ["cp_dose", "cp_time"] + mean_g_feats + mean_c_feats
ctl_group_data.columns = columns

#train_cons = train_feat[train_feat.index.isin(cons_train_index)].copy().reset_index(drop=True)
#test_cons = test_feat[test_feat.index.isin(cons_test_index)].copy().reset_index(drop=True)
#n_train_score = train_score[train_score.index.isin(cons_train_index)].copy().reset_index(drop=True)
#n_train_nonscore = train_nonscore[train_nonscore.index.isin(cons_train_index)].copy().reset_index(drop=True)

In [7]:
def regression(tr, ta, te):
    y_train = ta.sum(axis=1).copy()
    X_train = tr.copy()
    X_test = te.copy()
    for f in ["cp_dose", "cp_type"]:
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(tr[f]))
        X_train[f] = lbl.transform(list(X_train[f]))
        X_test[f] = lbl.transform(list(X_test[f])) 
    X_train.drop("sig_id", axis=1, inplace=True)
    X_test.drop("sig_id", axis=1, inplace=True)
    
    params = {'objective': 'regression', 'metric': 'rmse', 'boosting_type': 'gbdt', 'tree_learner': 'serial', 'learning_rate': 0.01, 
               "num_leaves": 10, 'random_seed':44, 'max_depth': 4} 
        
    n_folds=4
    skf=StratifiedKFold(n_splits = n_folds, shuffle=True, random_state=0)

    train_score_pred = np.zeros([X_train.shape[0]])
    test_score_pred = np.zeros(X_test.shape[0])
    for i , (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
        X_train2 = X_train.iloc[train_index,:]
        y_train2 = y_train.iloc[train_index]

        X_valid2 = X_train.iloc[test_index,:]
        y_valid2 = y_train.iloc[test_index]
        
        lgb_train = lgb.Dataset(X_train2, y_train2)
        lgb_eval = lgb.Dataset(X_valid2, y_valid2, reference=lgb_train)
        
        clf = lgb.train(params, lgb_train,valid_sets=[lgb_train, lgb_eval], 
               num_boost_round=10000,early_stopping_rounds=50,verbose_eval = 1000) 

        valid_predict = clf.predict(X_valid2, num_iteration = clf.best_iteration)
        train_score_pred[test_index] = valid_predict
        test_score_pred += clf.predict(X_test, num_iteration = clf.best_iteration) / n_folds
            
    print("score:", mean_squared_error(y_train, train_score_pred))
    return train_score_pred, test_score_pred

train_score_pred, test_score_pred = regression(train, targets, test)

Training until validation scores don't improve for 50 rounds
[1000]	training's rmse: 0.524983	valid_1's rmse: 0.57542
Early stopping, best iteration is:
[1182]	training's rmse: 0.517943	valid_1's rmse: 0.575035
Training until validation scores don't improve for 50 rounds
[1000]	training's rmse: 0.526143	valid_1's rmse: 0.57163
Early stopping, best iteration is:
[1071]	training's rmse: 0.52337	valid_1's rmse: 0.571491
Training until validation scores don't improve for 50 rounds
[1000]	training's rmse: 0.527321	valid_1's rmse: 0.571052
Early stopping, best iteration is:
[1642]	training's rmse: 0.503547	valid_1's rmse: 0.569874
Training until validation scores don't improve for 50 rounds
[1000]	training's rmse: 0.528824	valid_1's rmse: 0.561002
Early stopping, best iteration is:
[1591]	training's rmse: 0.50649	valid_1's rmse: 0.56025
score: 0.32397641910322417


In [8]:
train = pd.merge(train, ctl_group_data, on=["cp_time", "cp_dose"], how="left")
test = pd.merge(test, ctl_group_data, on=["cp_time", "cp_dose"], how="left")

for i in range(len(g_feats)):
    train["diff-g-"+str(i)] = train["g-"+str(i)] - train["mean-g-"+str(i)]
    test["diff-g-"+str(i)] = test["g-"+str(i)] - test["mean-g-"+str(i)]
    
for i in range(len(c_feats)):
    train["diff-c-"+str(i)] = train["c-"+str(i)] - train["mean-c-"+str(i)]
    test["diff-c-"+str(i)] = test["c-"+str(i)] - test["mean-c-"+str(i)]

In [9]:
categoricals = ["cp_dose"]

def encoding(tr, te):
    for f in categoricals:
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(tr[f]))
        tr[f] = lbl.transform(list(tr[f]))
        te[f] = lbl.transform(list(te[f])) 
        
    return tr, te

train, test = encoding(train, test)

# Feature engineering

In [10]:
def fe(df, remove_features):
    df.drop(remove_features, axis=1, inplace=True)
    return df

remove_features = ["cp_type" , "sig_id"] + mean_g_feats + mean_c_feats + g_feats + c_feats 
#for i in [i for i in train.columns if i != "sig_id"]:
#    if i not in remove_features and (n_train_feat[i].std() == 0):
#        remove_features.append(i)
        
train = fe(train, remove_features)
test = fe(test, remove_features)
train["score_sum"] = train_score_pred
test["score_sum"] = test_score_pred
    
print(train.shape, test.shape)

(23814, 875) (3982, 875)


# modelling

In [11]:
classifier = MultiOutputClassifier(XGBClassifier(tree_method='gpu_hist'))

clf = Pipeline([('classify', classifier)
               ])

params = {'classify__estimator__gamma': 3.6975,
          'classify__estimator__learning_rate': 0.0603,
          'classify__estimator__max_delta_step': 2.0706,
          'classify__estimator__max_depth': 10,
          'classify__estimator__min_child_weight': 31.5800,
          'classify__estimator__n_estimators': 166,
         }

clf.set_params(**params)

Pipeline(steps=[('classify',
                 MultiOutputClassifier(estimator=XGBClassifier(base_score=None,
                                                               booster=None,
                                                               colsample_bylevel=None,
                                                               colsample_bynode=None,
                                                               colsample_bytree=None,
                                                               gamma=3.6975,
                                                               gpu_id=None,
                                                               importance_type='gain',
                                                               interaction_constraints=None,
                                                               learning_rate=0.0603,
                                                               max_delta_step=2.0706,
                                                   

In [12]:
X = train.copy()
y = targets.drop("sig_id", axis=1).copy()
X_test = test.copy()

In [13]:
oof_preds = np.zeros(y.shape)
test_preds = np.zeros((test.shape[0], y.shape[1]))
oof_losses = []
mskf = MultilabelStratifiedKFold(n_splits=NFOLDS, random_state=SEED)
for fn, (trn_idx, val_idx) in enumerate(mskf.split(X, y)):
    print('Starting fold: ', fn)
    X_train, X_val = X.iloc[trn_idx,:], X.iloc[val_idx,:].to_numpy()
    y_train, y_val = y.iloc[trn_idx], y.iloc[val_idx].to_numpy()
    
    X_train = X_train[X_train.index.isin(cons_train_index)].to_numpy()
    y_train = y_train[y_train.index.isin(cons_train_index)].to_numpy()
    
    clf.fit(X_train, y_train)
    val_preds = clf.predict_proba(X_val) # list of preds per class
    val_preds = np.array(val_preds)[:,:,1].T # take the positive class
    oof_preds[val_idx] = val_preds
    
    loss = log_loss(np.ravel(y_val), np.ravel(val_preds))
    print(loss)
    oof_losses.append(loss)
    preds = clf.predict_proba(X_test)
    preds = np.array(preds)[:,:,1].T # take the positive class
    test_preds += preds / NFOLDS
    
print(oof_losses)
print('Mean OOF loss across folds', np.mean(oof_losses))
print('STD OOF loss across folds', np.std(oof_losses))

Starting fold:  0
0.01666543050581402
Starting fold:  1
0.016792976328319952
Starting fold:  2
0.016726157396588233
Starting fold:  3
0.01688796637086785
Starting fold:  4
0.016736985421047904
[0.01666543050581402, 0.016792976328319952, 0.016726157396588233, 0.01688796637086785, 0.016736985421047904]
Mean OOF loss across folds 0.01676190320452759
STD OOF loss across folds 7.491742286785465e-05


In [14]:
train_checkscore = targets.copy()
train_checkscore.iloc[:,1:] = oof_preds
train_checkscore.loc[train_checkscore.index.isin(noncons_train_index),target_feats] = 0
print('OOF log loss: ', log_loss(np.ravel(y), np.ravel(np.array(train_checkscore.iloc[:,1:]))))

OOF log loss:  0.016486723472810064


In [15]:
sub.iloc[:,1:] = test_preds
sub.loc[sub.index.isin(noncons_test_index),target_feats] = 0
sub.to_csv('submission.csv', index=False)